# Config Objects

In `omni-fig`, all arguments and settings are contained in the config object, no matter how they were specified (in a config file, through the terminal, or in python). The config object can have a nested structure to specify

This tutorial will discuss simple ways to create and manipulate config objects.

In [1]:
import omnifig as fig
from yaml import dump

In [2]:
def viz(config): # Utility function to print out config objects nicely
    print(dump(config.pythonize()), end='')

## Creating Config objects

Config objects can be created using `fig.get_config()`, which allows you to either pass a nested dictionary to specify the parameters directly.

In [3]:
fig.get_config?

In [15]:
raw_params = {'a': 1, 'b': 'test', 'c':{'x':123.2, 'y':'12'}, 'ls':['1', 123, {'inside':'a list'}]}

config1 = fig.get_config(**raw_params)
viz(config1)

a: 1
b: test
c:
  x: 123.2
  y: '12'
ls:
- '1'
- 123
- inside: a list


However, probably the most common way to create config objects is by loading one or multiple existing config files. When specifying multiple files, they are composed with the same multiple-inheritance rules as python classes. Usually, all config files of a project are registered when the project is initialized (more on this later), but for now we'll just manually register all configs in the `config/` directory. You can then view all of the configs that are registered.

In [9]:
fig.register_config_dir('config/')
print(list(fig.view_configs()))

['advanced', 'basic', 'data/large', 'data/small', 'data/_shared', 'power', 'small-change']


In [12]:
config2 = fig.get_config('basic')
viz(config2)

ancestors:
- basic
device: cheap-cpu
epochs: 25
favorite-device: <>device
model:
  _type: simple-model
  size: 10
name: basic
root: test/dir


In [13]:
config3 = fig.get_config('small-change', 'basic')
viz(config3)

ancestors:
- small-change
- basic
device: good-gpu
epochs: 25
favorite-device: <>device
model:
  _type: variant-model
  size: 10
  special-sauce: -2
name: updated-experiment
root: test/dir


Note that the `ancestors` key holds an ordered list of all the config files that were composed to create this config object.

## Pulling Information from the Config object

The main way you extract information from the config object is using the `pull()` function. By default, everytime you pull something from the config object, it prints a line of the result to the console so you always know what information your script actually got.

In [26]:
a = config1.pull('a')
b = config1.pull('b')
print(f'a is {a}')
print(f'b is {b}')

c = config1.pull('c', silent=True)
print(f'c is {c}')

| a: 1
| b: 'test'
a is 1
b is test
c is {'x': 123.2, 'y': '12'}


In [33]:
config1.pull('c.x')
config1.pull('ls.2.inside')

| c.x: 123.2
| ls.2.inside: 'a list'


'a list'

In [28]:
config1.pull('c.z', 'uh oh') # you can provide a default value

| z: 'uh oh' (by default)


'uh oh'

In [31]:
config1.pull('c.z', '<>c.x') # the prefix "<>" turns the string into an address
config1.pull('c.q', '<>c.r', '<>c.s', 'failed') # you can pass multiple default values

| z --> c.x: 123.2
| q --> r --> s: 'failed' (by default)


'failed'

In [32]:
config1.pull('c.a') # dictionaries by default check their parents if they can't find the parameter themselves.

| .a: 1


1